<a href="https://colab.research.google.com/github/sergiogf93/MetNumerics/blob/master/notebooks/09_FindingRoots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function

%matplotlib inline
import numpy
import matplotlib.pyplot as plt

# Trobar arrels

**OBJECTIU:** Trobar $x$ tal que $f(x) = 0$.

### Exemple:  Anualitat futura

Quan em puc jubilar?

$$ A = \frac{P}{(r / m)} \left[ \left(1 + \frac{r}{m} \right)^{m \cdot n} - 1 \right ] $$

$A$ valor total després de $n$ anys

$P$ pagament per cada període de composició

$m$ nombre de períodes de composició a l'any

$r$ tipus d'interès anual

$n$ nombre d'anys per la jubilació



Si em vull jubilar d'aquí a 20 anys quin tipus d'interès anual $r$ he de tenir?

Posem $P = \frac{\$18,000}{12} = \$1500, \quad m=12, \quad n=20$.

In [0]:
def total_value(P, m, r, n):
    """Total value of portfolio given parameters
    
    Based on following formula:
    
    A = \frac{P}{(r / m)} \left[ \left(1 + \frac{r}{m} \right)^{m \cdot n}
                - 1 \right ] 
    
    :Input:
     - *P* (float) - Payment amount per compounding period
     - *m* (int) - number of compounding periods per year
     - *r* (float) - annual interest rate
     - *n* (float) - number of years to retirement
     
     :Returns:
     (float) - total value of portfolio
     
    """
    
    return P / (r / float(m)) * ( (1.0 + r / float(m))**(float(m) * n)
                                 - 1.0)

P = 1500.0
m = 12
n = 20.0
    
r = numpy.linspace(0.05, 0.1, 100)
goal = 1e6

fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
axes.plot(r, total_value(P, m, r, n))
axes.plot(r, numpy.ones(r.shape) * goal, 'r--')
axes.set_xlabel("r (interest rate)")
axes.set_ylabel("A (total value)")
axes.set_title("When can I retire?")
axes.ticklabel_format(axis='y', style='sci', scilimits=(-1,1))
axes.set_xlim((0.05, 0.1))
axes.set_ylim((total_value(P, m, 0.05, n), total_value(P, m, 0.1, n)))
plt.show()

## Métode del punt fix

Com podem trobar la solució?

Podem intentar aïllar $r$:

$$ A = \frac{P}{(r / m)} \left[ \left(1 + \frac{r}{m} \right)^{m \cdot n} - 1 \right ] ~~~~ \Rightarrow ~~~~~$$

$$ r = \frac{P \cdot m}{A} \left[ \left(1 + \frac{r}{m} \right)^{m \cdot n} - 1 \right ] ~~~~ \Rightarrow ~~~~~$$

$$ r = g(r)$$

o bé:

$$ g(r) - r = 0$$

In [0]:
def g(P, m, r, n, A):
    """Reformulated minimization problem
    
    Based on following formula:
    
    g(r) = \frac{P \cdot m}{A} \left[ \left(1 + \frac{r}{m} \right)^{m \cdot n} - 1 \right ]
    
    :Input:
     - *P* (float) - Payment amount per compounding period
     - *m* (int) - number of compounding periods per year
     - *r* (float) - annual interest rate
     - *n* (float) - number of years to retirement
     - *A* (float) - total value after $n$ years
     
     :Returns:
     (float) - value of g(r)
     
    """
    
    return P * m / A * ( (1.0 + r / float(m))**(float(m) * n)
                                 - 1.0)

P = 1500.0
m = 12
n = 20.0
    
r = numpy.linspace(0.00, 0.1, 100)
goal = 1e6

fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
axes.plot(r, g(P, m, r, n, goal))
axes.plot(r, r, 'r--')
axes.set_xlabel("r (interest rate)")
axes.set_ylabel("$g(r)$")
axes.set_title("When can I retire?")
axes.set_ylim([0, 0.12])
axes.ticklabel_format(axis='y', style='sci', scilimits=(-1,1))
axes.set_xlim((0.00, 0.1))
axes.set_ylim((g(P, m, 0.00, n, goal), g(P, m, 0.1, n, goal)))
plt.show()

Probem un $r_0$ i comprovem en quina direcció hem d'anar...

1. $r_0 = 0.0800, \quad g(r_0) - r_0 = -0.009317550125425428$
1. $r_1 = 0.0850, \quad g(r_1) - r_1 = -0.00505763375972$
1. $r_2 = 0.0875, \quad g(r_2) - r_2 = -0.00257275331014$

Una mica tediós... Podem fer-ho amb codi:

In [0]:
r_values = numpy.linspace(0.08, 0.09, 11)

for r in r_values:
    print("r = ", r, "g(r) =", g(P, m, r, n, goal))
    print("Difference = ", numpy.abs(g(P, m, r, n, goal) - r))
    r = g(P, m, r, n, goal)

### Exemple 2:

Sigui $f(x) = x - e^{-x}$, troba $f(x) = 0$

Equivalent a $x = e^{-x}$ o $x = g(x)$ amb $g(x) = e^{-x}$

In [0]:
x = numpy.linspace(0.2, 1.0, 100)

fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
axes.plot(x, numpy.exp(-x), 'r')
axes.plot(x, x, 'b')
axes.set_xlabel("x")
axes.set_ylabel("f(x)")

x = 0.4
for steps in range(3):
    print("x = ", x, "Residual = ", numpy.abs(numpy.exp(-x) - x))
    x = numpy.exp(-x)
    axes.plot(x, numpy.exp(-x),'ko')
    axes.text(x+0.01, numpy.exp(-x)+0.01, steps+1, fontsize="15")

plt.show()

### Exemple 3:

Sigui $f(x) = \ln x + x$ troba $f(x) = 0$ o $x = -\ln x$.

Noteu que aquest problema és equivalent a $x = e^{-x}$.

In [0]:
  #Escriu un codi similar al de la cel·la anterior aplicat a aquest problema

In [0]:
#@title
x = numpy.linspace(0.1, 1.0, 100)

fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
axes.plot(x, -numpy.log(x), 'r')
axes.plot(x, x, 'b')
axes.set_xlabel("x")
axes.set_ylabel("f(x)")
axes.set_ylim([0, 1.5])

x = 0.5
for steps in range(3):
    print("x = ", x, "Residual = ", numpy.abs(numpy.log(x) + x))
    x = -numpy.log(x)
    axes.plot(x, -numpy.log(x),'ko')
    axes.text(x + 0.01, -numpy.log(x) + 0.01, steps+1, fontsize="15")
    
plt.show()

Aquests són problemes equivalents! Aquí passa alguna cosa...

## Anàlisi de la iteració del punt fix

Existència i unicitat de problemes del punt fix

*Existència:*

Suposem $g \in C[a, b]$, si la imatge $y = g(x)$ satisfà $y \in [a, b] \quad \forall \quad x \in [a, b]$ llavors $g$ té un punt fix $[a, b]$.

In [0]:
x = numpy.linspace(0.0, 1.0, 100)

# Plot function and intercept
fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
axes.plot(x, numpy.exp(-x), 'r')
axes.plot(x, x, 'b')
axes.set_xlabel("x")
axes.set_ylabel("f(x)")

# Plot domain and range
axes.plot(numpy.ones(x.shape) * 0.4, x, '--k')
axes.plot(numpy.ones(x.shape) * 0.8, x, '--k')
axes.plot(x, numpy.ones(x.shape) * numpy.exp(-0.4), '--k')
axes.plot(x, numpy.ones(x.shape) * numpy.exp(-0.8), '--k')

axes.set_xlim((0.0, 1.0))
axes.set_ylim((0.0, 1.0))

plt.show()

In [0]:
x = numpy.linspace(0.1, 1.0, 100)

fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
axes.plot(x, -numpy.log(x), 'r')
axes.plot(x, x, 'b')
axes.set_xlabel("x")
axes.set_ylabel("f(x)")
axes.set_xlim([0.1, 1.0])
axes.set_ylim([0.1, 1.0])

# Plot domain and range
axes.plot(numpy.ones(x.shape) * 0.4, x, '--k')
axes.plot(numpy.ones(x.shape) * 0.8, x, '--k')
axes.plot(x, numpy.ones(x.shape) * -numpy.log(0.4), '--k')
axes.plot(x, numpy.ones(x.shape) * -numpy.log(0.8), '--k')

plt.show()

In [0]:
r = numpy.linspace(0.06, 0.1, 100)
goal = 1e6

fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
axes.plot(r, g(P, m, r, n, goal))
axes.plot(r, r, 'r--')
axes.set_xlabel("r")
axes.set_ylabel("$g(r)$")
axes.set_xlim([0.06, 0.1])
axes.set_ylim([g(P, m, 0.06, n, goal), g(P, m, 0.1, n, goal)])
axes.ticklabel_format(axis='y', style='sci', scilimits=(-1,1))

axes.plot([0.08, 0.08], [g(P, m, 0.06, n, goal), g(P, m, 0.1, n, goal)], '--k')
axes.plot([0.09, 0.09], [g(P, m, 0.06, n, goal), g(P, m, 0.1, n, goal)], '--k')
axes.plot(r, numpy.ones(r.shape) * g(P, m, 0.08, n, goal), '--k')
axes.plot(r, numpy.ones(r.shape) * g(P, m, 0.09, n, goal), '--k')

plt.show()

*Unicitat:*

A més a més, si $g'(x)$ està definida a $x \in [a, b]$ i $\exists K < 1$ tal que

$$
    |g'(x)| \leq K < 1 \quad \forall \quad x \in (a,b)
$$

llavors $g$ té un únic punt fix $P \in [a,b]$

In [0]:
x = numpy.linspace(0.4, 0.8, 100)

fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
axes.plot(x, numpy.abs(-numpy.exp(-x)), 'r')
axes.plot(x, numpy.ones(x.shape), 'k--')
axes.set_xlabel("x")
axes.set_ylabel("f(x)")
axes.set_ylim((0.0, 1.1))
plt.show()

*Convergència asimptòtica*: Comportament de les iteracions de punt fix

$$x_{k+1} = g(x_k)$$

Si $\exists ~ x^\ast$ t.q. $x^\ast = g(x^\ast)$ (i.e. $x^\ast$ és un punt fix), llavors definim

$$
    x_k = x^\ast + e_k \quad \quad x_{k+1} = x^\ast + e_{k+1}
$$
i
$$
    x^\ast + e_{k+1} = g(x^\ast + e_k)
$$

Fem el desenvolupament de Taylor  $g$ al voltant de $x^\ast$:

$$
    g(x) = g(x^\ast) + g'(x^\ast) (x - x^\ast) + \frac{g''(x^\ast)}{2!} (x - x^\ast)^2 + \mathcal{O}((x - x^\ast)^3)
$$

Evaluem la sèrie a $x_k = x^\ast + e_k$ i trobem

$$
    g(x^\ast + e_k) = g(x^\ast) + g'(x^\ast) e_k + \frac{g''(x^\ast) e_k^2}{2} + \mathcal{O}(e_k^3)
$$

així doncs, per la definició anterior $x^\ast + e_{k+1} = g(x^\ast + e_k)$ tenim

$$
    x^\ast + e_{k+1} = g(x^\ast) + g'(x^\ast) e_k + \frac{g''(x^\ast) e_k^2}{2} + \mathcal{O}(e_k^3)
$$

Nota que com $x^* = g(x^*)$ aquests termes es cancel·len i

$$e_{k+1} = g'(x^*) e_k + \frac{g''(x^*) e_k^2}{2}$$

Així doncs, si $|g'(x^*)| \leq K < 1$ podem concloure que

$$|e_{k+1}| = K |e_k|$$

la qual cosa mostra la convergència. Noteu també que $K$ està relacionat amb $|g'(x^*)|$.

### Convergència dels métodes iteratius

Donat un mètode iteratiu amb:

$$|e_{k+1}| = C |e_k|^n$$

Si $C < 1$ i:
 - $n=1$ el mètode és **linealment convergent**
 - $n=2$ el mètode és **quadràticament convergent**
 - $n > 1$ el mètode s'en diu **superlinealment convergent**

Si $C > 1$ llavors el mètode és **divergent**



### Revisitem els exemples
$g(x) = e^{-x}$ amb $x^* \approx 0.56$
 
   $$|g'(x^*)| = |-e^{-x^*}| \approx 0.56$$
   

$g(x) = - \ln x \quad \text{amb} \quad x^* \approx 0.56$

   $$|g'(x^*)| = \frac{1}{|x^*|} \approx 1.79$$
   

$$
    r = g(r) = \frac{P \cdot m}{A} \left[ \left(1 + \frac{r}{m} \right)^{m \cdot n} - 1 \right ]
$$

In [0]:
import sympy
r, P, m, A, n = sympy.symbols('r P m A n')
g = P * m / A * ((1 + r /m)**(m * n) - 1)
g_prime = g.diff(r)
r_star = 0.08985602484084668
print("g'(r) = ", g_prime)
print("g'(r*) = ", g_prime.subs({P: 1500.0, m: 12, n:20, A: 1e6, r: r_star}))

f = sympy.lambdify(r, g_prime.subs({P: 1500.0, m: 12, n:20, A: 1e6}))
fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
r = numpy.linspace(-0.01, 0.1, 100)
axes.plot(r, f(r))
axes.plot(r, numpy.ones(r.shape), 'k--')
axes.plot(r_star, f(r_star), 'ro')
axes.plot(0.0, f(0.0), 'ro')
axes.set_xlim((-0.01, 0.1))
axes.set_xlabel("$r$")
axes.set_ylabel("$g'(r)$")
plt.show()

Veiem que al voltant de $r^*$ el métode divergeix.

## Mètodes millors per trobar arrels/optimització

Si $x^*$ és un punt fix de $g(x)$ llavors $x^*$ és una  *arrel* de $f(x^*) = g(x^*) - x^*$ t.q. $f(x^*) = 0$.

Per exemple:

$$f(r) = r - \frac{m P}{A} \left [ \left (1 + \frac{r}{m} \right)^{m n} - 1 \right ] =0 $$

o bé

$$f(r) = A - \frac{m P}{r} \left [ \left (1 + \frac{r}{m} \right)^{m n} - 1 \right ] =0 $$

## Mètodes clàssics
- Bisecció (convergència lineal)
- Mètode de Newton (convergència quadràtica)
- Mètode de les secants (convergència superlineal)
 
## Mètodes combinats
 - RootSafe (Newton + Bisecció)
 - Mètode de Brent (Secants + Bisecció)

### Bracketing i Bisecció

Un **bracket** és un interval $[a,b]$ que conté exactament un zero o un mínim/màxim d'interès.

En cas d'un zero, el bracket ha de satisfer:
$$
    \text{sign}(f(a)) \neq \text{sign}(f(b)).
$$

I en el cas d'un mínim o un màxim: 
$$
    \text{sign}(f'(a)) \neq \text{sign}(f'(b))
$$

**Teorema**:  

Sigui
$$
    f(x) \in C[a,b] \quad \text{i} \quad \text{sign}(f(a)) \neq \text{sign}(f(b))
$$

llavors existeix un nombre $c$:
$$
    c \in (a,b) \quad \text{t.q.} \quad f(c) = 0.
$$
(la demostració fa servir el teorema del valor mig)

In [0]:
P = 1500.0
m = 12
n = 20.0
A = 1e6
r = numpy.linspace(0.05, 0.1, 100)
f = lambda r, A, m, P, n: A - m * P / r * ((1.0 + r / m)**(m * n) - 1.0)

fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
axes.plot(r, f(r, A, m, P, n), 'b')
axes.plot(r, numpy.zeros(r.shape),'r--')
axes.set_xlabel("r (%)")
axes.set_ylabel("f(r)")
axes.ticklabel_format(axis='y', style='sci', scilimits=(-1,1))

a = 0.075
b = 0.095
axes.plot(a, f(a, A, m, P, n), 'ko')
axes.plot([a, a], [0.0, f(a, A, m, P, n)], 'k--')
axes.plot(b, f(b, A, m, P, n), 'ko')
axes.plot([b, b], [f(b, A, m, P, n), 0.0], 'k--')

plt.show()

Els algoritmes bàsics de bracketing consisteixen en reduir el bracket assegurant-nos que l'arrel (o el mínim/màxim) romangui a l'interior del bracket.

De quines maneres podem reduir el bracket de manera que els extrems convergeixin a l'arrel/mínim/màxim?

#### Algoritme de la bisecció

Donat un bracket $[a,b]$ i una funció $f(x)$
1. Inicialitzem el bracket
2. Iterem
  1. Partim el bracket per la meitat i comprovem en quina de les dues parts està l'arrel.
  2. Considerem com a nou bracket aquell que conté l'arrel.
  3. Si el valor de la funció al punt mig del bracket és suficientment proper a 0, aturem l'algoritme.

In [0]:
P = 1500.0
m = 12
n = 20.0
A = 1e6
r = numpy.linspace(0.05, 0.11, 100)
f = lambda r, A=A, m=m, P=P, n=n: A - m * P / r * ((1.0 + r / m)**(m * n) - 1.0)

# Initialize bracket
a = 0.07
b = 0.10

# Setup figure to plot convergence
fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
axes.plot(r, f(r, A, m, P, n), 'b')
axes.plot(r, numpy.zeros(r.shape),'r--')
axes.set_xlabel("r (%)")
axes.set_ylabel("f(r)")
# axes.set_xlim([0.085, 0.091])
axes.ticklabel_format(axis='y', style='sci', scilimits=(-1,1))
axes.plot(a, f(a, A, m, P, n), 'ko')
axes.plot([a, a], [0.0, f(a, A, m, P, n)], 'k--')
axes.plot(b, f(b, A, m, P, n), 'ko')
axes.plot([b, b], [f(b, A, m, P, n), 0.0], 'k--')

# Algorithm parameters
TOLERANCE = 1e-4
MAX_STEPS = 1000

# Initialize loop
delta_x = b - a
c = a + delta_x / 2.0
f_a = f(a)
f_b = f(b)
f_c = f(c)

# Escriu la implementació del mètode de la bisecció fins que s'arribi a ||f(c)|| < TOLERANCE, o el nombre d'iteracions sigui igual a MAX_STEPS

In [0]:
#@title
P = 1500.0
m = 12
n = 20.0
A = 1e6
r = numpy.linspace(0.05, 0.11, 100)
f = lambda r, A=A, m=m, P=P, n=n: A - m * P / r * ((1.0 + r / m)**(m * n) - 1.0)

# Initialize bracket
a = 0.07
b = 0.10

# Setup figure to plot convergence
fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
axes.plot(r, f(r, A, m, P, n), 'b')
axes.plot(r, numpy.zeros(r.shape),'r--')
axes.set_xlabel("r (%)")
axes.set_ylabel("f(r)")
# axes.set_xlim([0.085, 0.091])
axes.ticklabel_format(axis='y', style='sci', scilimits=(-1,1))
axes.plot(a, f(a, A, m, P, n), 'ko')
axes.plot([a, a], [0.0, f(a, A, m, P, n)], 'k--')
axes.plot(b, f(b, A, m, P, n), 'ko')
axes.plot([b, b], [f(b, A, m, P, n), 0.0], 'k--')

# Algorithm parameters
TOLERANCE = 1e-4
MAX_STEPS = 1000

# Initialize loop
delta_x = b - a
c = a + delta_x / 2.0
f_a = f(a)
f_b = f(b)
f_c = f(c)
# Loop until we reach the TOLERANCE or we take MAX_STEPS
for step in range(1, MAX_STEPS + 1):
    
    # Plot iteration
    axes.plot(c, f_c,'kx')
    axes.text(c, f_c, str(step + 1), fontsize="15")

    # Check tolerance - Could also check the size of delta_x
    # We check this first as we have already initialized the values
    # in c and f_c
    if numpy.abs(f_c) < TOLERANCE:
        break

    if numpy.sign(f_a) != numpy.sign(f_c):
        b = c
        f_b = f_c
    else:
        a = c
        f_a = f_c
    delta_x = b - a
    c = a + delta_x / 2.0
    f_c = f(c)
        
if step == MAX_STEPS:
    print("Reached maximum number of steps!")
else:
    print("Success!")
    print("  x* = %s" % c)
    print("  f(x*) = %s" % f(c))
    print("  number of steps = %s" % step)

#### Convergència de la Bisecció

En general tenim:
$$
    |e_{k+1}| = C |e_k|^n
$$
i necessitem que $C < 1$ i $n > 0$.

Sigui $\Delta x_k$ l'amplada del $k$th bracket podem estimar l'error com
$$
    e_k \approx \Delta x_k
$$
així doncs
$$
    e_{k+1} \approx \frac{1}{2} \Delta x_k.
$$
per tant
$$
    |e_{k+1}| = \frac{1}{2} |e_k|
$$
veiem doncs que el mètode és linealment convergent.

### Mètode de Newton (Newton-Raphson)
- Donat un bracket, la bisecció garanteix la convergència lineal a l'arrel.
- Tot i això, la bisecció no utilitza pràcticament cap informació sobre $f(x)$ a part del signe en un punt.

**Idea bàsica**: Donada $f(x)$ i $f'(x)$ fem una aproximació lineal a $f(x)$ localment i utilitzem el tall de la reacta a l'eix de les $x$ per fer una predicció d'on pot estar $x^*$.


Donada una ubicació $x_k$, tenim $f(x_k)$ i $f'(x_k)$.

L'equació de la recta tangent a $f(x)$ per $x_k$ serà:

$$y = f'(x_k) x + b$$

Trobem el tall amb l'eix de les $y$'s $b$:'

$$f(x_k) = f'(x_k) x_k + b$$

$$b = f(x_k) - f'(x_k) x_k$$

i simplifiquem:

$$y = f'(x_k) x + f(x_k) - f'(x_k) x_k$$

$$y = f'(x_k) (x - x_k) + f(x_k)$$

Ara trobem la intersecció de la recta amb l'eix de les $x$'s (i.e. quan $y = 0$) i utilitzem el corresponent valor de $x$ per obtenir $x_{k+1}$

$$
    0 = f'(x_k) (x_{k+1}-x_k) + f(x_k)
$$

$$
    \boxed{x_{k+1} = x_k-\frac{f(x_k)}{f'(x_k)}}
$$

Una manera alternativa de derivar el mètode de Newton-Raphson utilitza el desenvolupament de Taylor. Si ampliem l'equació $f(x)$ al voltant del valor actual de la iteració $x_k$:


$$
    f(x) = f(x_k) + f'(x_k) (x - x_k) + \frac{f''(x_k)}{2!} (x - x_k)^2 + \mathcal{O}((x-x_k)^3)
$$

Sigui $\Delta_k$ l'increment per obtenir el següent valor de la iteració $x_{k+1}$:

$$
    x_{k+1} = x_k + \Delta x_k
$$

i evaluem el valor de $f(x)$ a $x_{k+1}$:
$$
    f(x_{k+1}) = f(x_k) + f'(x_k) \Delta x_k + \frac{f''(x_k)}{2!} \Delta x_k^2 + \mathcal{O}(\Delta x_k^3)
$$

Ara considerem que $x_{k+1} = x^\ast$, en aquest cas
$$
    0 = f(x_k) + f'(x_k) \Delta x_k + \frac{f''(x_k)}{2!} \Delta x_k^2 + \mathcal{O}(\Delta x_k^3)
$$

Si treiem els termes d'ordre superior obtenim:

$$
    \Delta x_k = - \frac{f(x_k)}{f'(x_k)}
$$

si $f \in \mathbb R$ obtenim

$$
    \boxed{x_{k+1} = x_k - \frac{f(x_k)}{f'(x_k)}}
$$

In [0]:
P = 1500.0
m = 12
n = 20.0
A = 1e6
r = numpy.linspace(0.05, 0.11, 100)
f = lambda r, A=A, m=m, P=P, n=n: \
        A - m * P / r * ((1.0 + r / m)**(m * n) - 1.0)
f_prime = lambda r, A=A, m=m, P=P, n=n: \
            -P*m*n*(1.0 + r/m)**(m*n)/(r*(1.0 + r/m)) \
                + P*m*((1.0 + r/m)**(m*n) - 1.0)/r**2

# Initial guess
x_k = 0.06

# Setup figure to plot convergence
fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
axes.plot(r, f(r), 'b')
axes.plot(r, numpy.zeros(r.shape),'r--')

# Plot x_k point
axes.plot([x_k, x_k], [0.0, f(x_k)], 'k--')
axes.plot(x_k, f(x_k), 'ko')
axes.text(x_k, -5e4, "$x_k$", fontsize=16)
axes.plot(x_k, 0.0, 'xk')
axes.text(x_k, f(x_k) + 2e4, "$f(x_k)$", fontsize=16)
axes.plot(r, f_prime(x_k) * (r - x_k) + f(x_k), 'k')

# Plot x_{k+1} point
x_k = x_k - f(x_k) / f_prime(x_k)
axes.plot([x_k, x_k], [0.0, f(x_k)], 'k--')
axes.plot(x_k, f(x_k), 'ko')
axes.text(x_k, 1e4, "$x_{k+1}$", fontsize=16)
axes.plot(x_k, 0.0, 'xk')
axes.text(0.0873, f(x_k) - 2e4, "$f(x_{k+1})$", fontsize=16)

axes.set_xlabel("r (%)")
axes.set_ylabel("f(r)")
axes.set_title("Newton-Raphson Steps")
axes.ticklabel_format(axis='y', style='sci', scilimits=(-1,1))

plt.show()

### Algoritme per Newton-Raphson

1. Inicialitzar $x_k$
1. Comencem la iteració
  1. Calcular $f(x_k)$ i $f'(x_k)$
  1. Obtenir $x_{k+1}$ a partir de $f(x_k)$ i $f'(x_k)$
  1. Comprovar la condició per aturar la iteració

In [0]:
P = 1500.0
m = 12
n = 20.0
A = 1e6
r = numpy.linspace(0.05, 0.11, 100)
f = lambda r, A=A, m=m, P=P, n=n: \
        A - m * P / r * ((1.0 + r / m)**(m * n) - 1.0)
f_prime = lambda r, A=A, m=m, P=P, n=n: \
            -P*m*n*(1.0 + r/m)**(m*n)/(r*(1.0 + r/m)) \
                + P*m*((1.0 + r/m)**(m*n) - 1.0)/r**2

# Algorithm parameters
MAX_STEPS = 200
TOLERANCE = 1e-4
        
# Initial guess
x_k = 0.06

# Implementa el métode de Newton-Raphson per la funció f

In [0]:
#@title
P = 1500.0
m = 12
n = 20.0
A = 1e6
r = numpy.linspace(0.05, 0.11, 100)
f = lambda r, A=A, m=m, P=P, n=n: \
        A - m * P / r * ((1.0 + r / m)**(m * n) - 1.0)
f_prime = lambda r, A=A, m=m, P=P, n=n: \
            -P*m*n*(1.0 + r/m)**(m*n)/(r*(1.0 + r/m)) \
                + P*m*((1.0 + r/m)**(m*n) - 1.0)/r**2

# Algorithm parameters
MAX_STEPS = 200
TOLERANCE = 1e-4
        
# Initial guess
x_k = 0.06

# Setup figure to plot convergence
fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
axes.plot(r, f(r), 'b')
axes.plot(r, numpy.zeros(r.shape),'r--')

for n in range(1, MAX_STEPS + 1):
    axes.plot(x_k, f(x_k),'kx')
    axes.text(x_k, f(x_k), str(n), fontsize="15")
    x_k = x_k - f(x_k) / f_prime(x_k)
    if numpy.abs(f(x_k)) < TOLERANCE:
        break
        
if n == MAX_STEPS:
    print("Reached maximum number of steps!")
else:
    print("Success!")
    print("  x* = %s" % x_k)
    print("  f(x*) = %s" % f(x_k))
    print("  number of steps = %s" % n)

axes.set_xlabel("r (%)")
axes.set_ylabel("f(r)")
axes.set_title("Newton-Raphson Steps")
axes.ticklabel_format(axis='y', style='sci', scilimits=(-1,1))

plt.show()

### Convergència asimptòtica del mètode de Newton

Per una arrel simple, sigui $g(x) = x - \frac{f(x)}{f'(x)}$, llavors

$$x_{k+1} = g(x_k)$$

Definim els errors i les iteracions:

$$x_{k+1} = x^* + e_{k+1} \quad \quad x_k = x^* + e_k$$

El desenvolupament general de Taylor:

$$
    x^* + e_{k+1} = g(x^* + e_k) = g(x^*) + g'(x^*) e_k + \frac{g''(x^*) e_k^2}{2!} + \mathcal{O}(e_k^3)
$$

Noteu que $x^*$ i $g(x^*)$ es cancel·len:

$$e_{k+1} = g'(x^*) e_k + \frac{g''(x^*) e_k^2}{2!} + \ldots$$

Què passa amb $g'(x^*)$?  

$$\begin{aligned}
    g(x) &= x - \frac{f(x)}{f'(x)} \\
    g'(x) & = 1 - \frac{f'(x)}{f'(x)} + \frac{f(x) f''(x)}{(f'(x))^2} = \frac{f(x) f''(x)}{(f'(x))^2}
\end{aligned}$$

evaluat a $x = x^*$ dóna

$$
    g'(x^*) = \frac{f(x^*)f''(x^*)}{f'(x^*)^2} = 0
$$

donat que $f(x^\ast) = 0$ (assumint que $f''(x^\ast)$ i $f'(x^\ast)$ tenen bon comportament).

Tornant al desenvolupament, tornem a tenir:

$$
    e_{k+1} = g'(x^*) e_k + \frac{g''(x^*) e_k^2}{2!} + \ldots
$$

que dóna

$$
    e_{k+1} = \frac{g''(x^*) e_k^2}{2!} + \ldots
$$

$$
    |e_{k+1}| < \left | \frac{g''(x^*)}{2!} \right | |e_k|^2
$$

Així doncs, el métode de Newton és quadràticament convergent i la constant és controlada per la segona derivada.

Pel cas d'arrels múltiples (e.g. $f(x) = (x-1)^2$) la cosa no funciona tant bé. Per què?

  ### Exemple:
$f(x) = \sin (2 \pi x)$

$$x_{k+1} = x_k - \frac{\sin (2 \pi x)}{2 \pi \cos (2 \pi x)}= x_k - \frac{1}{2 \pi} \tan (2 \pi x)$$

In [0]:
x = numpy.linspace(0, 2, 1000)
f = lambda x: numpy.sin(2.0 * numpy.pi * x)
f_prime = lambda x: 2.0 * numpy.pi * numpy.cos(2.0 * numpy.pi * x)

fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
axes.plot(x, f(x),'b')
axes.plot(x, f_prime(x), 'r')
axes.set_xlabel("x")
axes.set_ylabel("y")
axes.set_title("Comparison of $f(x)$ and $f'(x)$")
axes.set_ylim((-2,2))
axes.set_xlim((0,2))
axes.plot(x, numpy.zeros(x.shape), 'k--')

x_k = 0.3
axes.plot([x_k, x_k], [0.0, f(x_k)], 'ko')
axes.plot([x_k, x_k], [0.0, f(x_k)], 'k--')
axes.plot(x, f_prime(x_k) * (x - x_k) + f(x_k), 'k')


x_k = x_k - f(x_k) / f_prime(x_k)
axes.plot([x_k, x_k], [0.0, f(x_k)], 'ko')
axes.plot([x_k, x_k], [0.0, f(x_k)], 'k--')

plt.show()

In [0]:
x = numpy.linspace(0, 2, 1000)
f = lambda x: numpy.sin(2.0 * numpy.pi * x)
x_kp = lambda x: x - 1.0 / (2.0 * numpy.pi) * numpy.tan(2.0 * numpy.pi * x)

fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
axes.plot(x, f(x),'b')
axes.plot(x, x_kp(x), 'r')
axes.set_xlabel("x")
axes.set_ylabel("y")
axes.set_title("Comparison of $f(x)$ and $f'(x)$")
axes.set_ylim((-2,2))
axes.set_xlim((0,2))
axes.plot(x, numpy.zeros(x.shape), 'k--')

plt.show()

### Conques d'atracció

Donat un punt $x_0$ podem determinar si el mètode de Newton-Raphson convergeix?

Una *conca d'atracció* $X$ per mètodes de Newton es defineix com el conjunt tal que  $\forall x \in X$ les iteracions de Newton convergeixen. Aquests conjunts no són trivials de determinar i fins i tot per funcions simples es poden obtenir fractals.

A sota es mostren els resultats per dues equacions:

1. $f(x) = x^3 - 1$
2. Equació de Kepler $\theta - e \sin \theta = M$

In [0]:
f = lambda x: x**3 - 1
f_prime = lambda x: 3 * x**2

N = 1001
x = numpy.linspace(-2, 2, N)
X, Y = numpy.meshgrid(x, x)
R = X + 1j * Y

for i in range(30):
    R = R - f(R) / f_prime(R)
    
fig = plt.figure()
fig.set_figwidth(fig.get_figwidth() * 2)
fig.set_figheight(fig.get_figheight() * 2)
axes = fig.add_subplot(1, 1, 1, aspect='equal')
axes.contour(X, Y, R)
axes.set_xlabel("Real")
axes.set_ylabel("Imaginary")
axes.set_title("Basin of Attraction for $f(x) = x^3 - 1$")
plt.show()

In [0]:
def f(theta, e=0.083, M=1):
    return theta - e * numpy.sin(theta) - M
def f_prime(theta, e=0.083):
    return 1 - e * numpy.cos(theta)

N = 1001
x = numpy.linspace(-30.5, -29.5, N)
y = numpy.linspace(-17.5, -16.5, N)
X, Y = numpy.meshgrid(x, y)
R = X + 1j * Y

for i in range(30):
    R = R - f(R) / f_prime(R)
    
fig = plt.figure()
fig.set_figwidth(fig.get_figwidth() * 2)
fig.set_figheight(fig.get_figheight() * 2)
axes = fig.add_subplot(1, 1, 1, aspect='equal')
axes.contour(X, Y, R)
axes.set_xlabel("Real")
axes.set_ylabel("Imaginary")
axes.set_title("Basin of Attraction for $f(x) = x - e \sin x - M$")

#### Altres problemes amb el mètode

Hem de donar $f(x)$ i $f'(x)$, podría no computacionalment car.
 
Exemple:  $f(r) = A - \frac{m P}{r} \left[ \left(1 + \frac{r}{m} \right )^{m n} - 1\right]$

Podríem utilitzar (`sympy`)

### Mètode de les secants

Hi ha alguna manera d'obtenir la convergència del mètode de Newton però sense les derivades extra? De quina manera podríem modificar el mètode de Newton?


Donades $x_k$ i $x_{k-1}$ representem la derivada amb l'aproximació

$$f'(x) \approx \frac{f(x_k) - f(x_{k-1})}{x_k - x_{k-1}}$$

Combinat amb el mètode de Newton:

$$\boxed{x_{k+1} = x_k - \frac{f(x_k) (x_k - x_{k-1}) }{f(x_k) - f(x_{k-1})}}$$

Això dóna convergència. No arriba a la convergència quadràtica, l'exponent és $\approx 1.7$.

També podem obtenir el resultat considerant la recta que passa per dons punts de la funció i trobant el tall amb l'eix d'abscisses:

$$(x_k, f(x_k)) ~~~~~ (x_{k-1}, f(x_{k-1})$$

$$y = \frac{f(x_k) - f(x_{k-1})}{x_k - x_{k-1}} (x - x_k) + b$$

$$b = f(x_{k-1}) - \frac{f(x_k) - f(x_{k-1})}{x_k - x_{k-1}} (x_{k-1} - x_k)$$

$$ y = \frac{f(x_k) - f(x_{k-1})}{x_k - x_{k-1}} (x - x_k) + f(x_k)$$

Aïllem $x_{k+1}$:

$$0 = \frac{f(x_k) - f(x_{k-1})}{x_k - x_{k-1}} (x_{k+1} - x_k) + f(x_k)$$

$$x_{k+1} = x_k - \frac{f(x_k)  (x_k - x_{k-1})}{f(x_k) - f(x_{k-1})}$$

In [0]:
P = 1500.0
m = 12
n = 20.0
A = 1e6
r = numpy.linspace(0.05, 0.11, 100)
f = lambda r, A=A, m=m, P=P, n=n: \
        A - m * P / r * ((1.0 + r / m)**(m * n) - 1.0)

# Initial guess
x_k = 0.07
x_km = 0.06

fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
axes.plot(r, f(r), 'b')
axes.plot(r, numpy.zeros(r.shape),'r--')

axes.plot(x_k, 0.0, 'ko')
axes.plot(x_k, f(x_k), 'ko')
axes.plot([x_k, x_k], [0.0, f(x_k)], 'k--')
axes.plot(x_km, 0.0, 'ko')
axes.plot(x_km, f(x_km), 'ko')
axes.plot([x_km, x_km], [0.0, f(x_km)], 'k--')

axes.plot(r, (f(x_k) - f(x_km)) / (x_k - x_km) * (r - x_k) + f(x_k), 'k')
x_kp = x_k - (f(x_k) * (x_k - x_km) / (f(x_k) - f(x_km)))
axes.plot(x_kp, 0.0, 'ro')
axes.plot([x_kp, x_kp], [0.0, f(x_kp)], 'r--')
axes.plot(x_kp, f(x_kp), 'ro')

axes.set_xlabel("r (%)")
axes.set_ylabel("f(r)")
axes.set_title("Secant Method")
axes.ticklabel_format(axis='y', style='sci', scilimits=(-1,1))

plt.show()

#### Algoritme pel mètode de les secants

Donada $f(x)$, un bracket $[a,b]$, una `TOLERANCE`, i `MAX_STEPS` (noteu que necessitem dos punts per començar).

1. Inicialitzem $x_1 = a$, $x_2 = b$, $f_1 = f(x_1)$, i $f_2 = f(x_2)$
2. Iterem fins assolir `MAX_STEPS` o obtenir `TOLERANCE`
   1. Calcular $x_{k+1}$
   2. Comprovar la condició per acabar la iteració
   3. Actualitzar els paràmetres $x_1$, $x_2$, $f_1 = f(x_1)$ i $f_2(x_2)$

In [0]:
P = 1500.0
m = 12
n = 20.0
A = 1e6
r = numpy.linspace(0.05, 0.11, 100)
f = lambda r, A=A, m=m, P=P, n=n: \
        A - m * P / r * ((1.0 + r / m)**(m * n) - 1.0)
f_prime = lambda r, A=A, m=m, P=P, n=n: \
            -P*m*n*(1.0 + r/m)**(m*n)/(r*(1.0 + r/m)) \
                + P*m*((1.0 + r/m)**(m*n) - 1.0)/r**2

# Algorithm parameters
MAX_STEPS = 50
TOLERANCE = 1e-4
        
# Initial bracket
x_k = 0.07
x_km = 0.06

# Implementa el mètode de les secants

In [0]:
#@title
P = 1500.0
m = 12
n = 20.0
A = 1e6
r = numpy.linspace(0.05, 0.11, 100)
f = lambda r, A=A, m=m, P=P, n=n: \
        A - m * P / r * ((1.0 + r / m)**(m * n) - 1.0)
f_prime = lambda r, A=A, m=m, P=P, n=n: \
            -P*m*n*(1.0 + r/m)**(m*n)/(r*(1.0 + r/m)) \
                + P*m*((1.0 + r/m)**(m*n) - 1.0)/r**2

# Algorithm parameters
MAX_STEPS = 50
TOLERANCE = 1e-4
        
# Initial bracket
x_k = 0.07
x_km = 0.06

# Setup figure to plot convergence
fig = plt.figure()
axes = fig.add_subplot(1, 1, 1)
axes.plot(r, f(r), 'b')
axes.plot(r, numpy.zeros(r.shape),'r--')

for n in range(1, MAX_STEPS + 1):
    axes.plot(x_k, f(x_k), 'o')
    axes.text(x_k + 0.0025, f(x_k), n, fontsize="15")
    x_kp = x_k - f(x_k) * (x_k - x_km) / (f(x_k) - f(x_km))
    x_km = x_k
    x_k = x_kp
    print("Residual = ", numpy.abs(f(x_k)))
    if numpy.abs(f(x_k)) < TOLERANCE:
        break
        
if n == MAX_STEPS:
    print("Reached maximum number of steps!")
else:
    print("Success!")
    print("  x* = %s" % x_k)
    print("  f(x*) = %s" % f(x_k))
    print("  number of steps = %s" % n)

axes.set_xlabel("r (%)")
axes.set_ylabel("f(r)")
axes.set_title("Secant Method")
axes.ticklabel_format(axis='y', style='sci', scilimits=(-1,1))

plt.show()

#### Comentaris

- El mètode de les secants presentat és equivalent a la interpolació lineal.
- Es pot fer interpolació d'ordres més elevats per fer mètodes de les secants d'ordre superior.
- La convergència no és del tot quadràtica.
- No es garanteix la convergència.
- No es preserven els brackets.
- Pot ser tant bo com el mètode de Newton si el valor inicial és adequat.


### Mètodes híbrids


#### Objectius
1. Robustesa:  Donat un bracket $[a,b]$, mantenir el bracket
1. Eficiència:  Utilitzar mètodes de convergència superlineal quan sigui possible

#### Opcions
 - Mètodes que requereixen $f'(x)$
   - NewtSafe (RootSafe, Numerical Recipes)
   - Newton's Method dins un bracket, Bisection en cas contrari
 - Mètodes que no requereixen $f'(x)$
   - Brent's Algorithm (zbrent, Numerical Recipes)
     - Combinar bisecció, secants i interpolació quadràtica inversa
   - `scipy.optimize` package

## Scipy

Scipy conté eines per fer optimització!

In [0]:
import scipy.optimize as optimize
print(optimize.golden(f, brack=(0.2, 0.25, 0.5)))